In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
from IPython.display import display
import time
import ratelimit

# Maximum number of requests in a `LIMIT_PERIOD` period.
LIMIT_CALLS: int = 1
# Duration of a period, in seconds.
LIMIT_PERIOD: int = 2

@ratelimit.sleep_and_retry
@ratelimit.limits(calls=LIMIT_CALLS, period=LIMIT_PERIOD)
def check_autovit_limit() -> None:
    """
    Utility function which forces requests to Autovit to respect rate limits.
    This function should be used as a "guard", simply call it before you make a
    request.
    """
    pass

def awesomeScrap(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    soup = BeautifulSoup(mystr, features="html.parser")
    body = soup.body
    siteWrap = body.find("div", {"id":"siteWrap"})
    flexContainerMain = siteWrap.find("div", {"class":"flex-container-main"})
    flexContainerMainLeft = flexContainerMain.find("div", {"class":"flex-container-main__left"})
    flexContainerMainRight = flexContainerMain.find("div", {"class":"flex-container-main__right"})

    offerContentAsside = flexContainerMainRight.find("div", {"class":"offer-content__aside"})
    offerSummary = offerContentAsside.find("div",{"class":"offer-summary"})
    priceWrapper = offerSummary.find("div", {"class":"price-wrapper"})
    offerPrice = priceWrapper.find("div", {"class":"offer-price"})
    offerContent = flexContainerMainLeft.find("div", {"class":"offer-content offer-content--secondary"})
    offerContentRaw = offerContent.find("div", {"class":"offer-content__row om-offer-main"})
    offerContentRawMain = offerContentRaw.find("div", {"class":"offer-content__main-column"})
    parametersArea = offerContentRawMain.find("div", {"class":"parametersArea"})
    parameters = parametersArea.find("div", {"id": "parameters"})
    myDic = {'Pret':offerPrice['data-price']}
    for listCaract in parameters.find_all_next("ul"):
        for values in listCaract.find_all_next("li"):
            valuesParam =values.find("div",{"class":"offer-params__value"})
            valuesName = values.find("span",{"class":"offer-params__label"})
            try:
                if valuesParam.a != None:
                    myDic[valuesName.string] = valuesParam.a.string.strip()
                else:
                    myDic[valuesName.string] = valuesParam.string.strip()
            except:
                continue

    return myDic

def main():

    links = pd.read_csv("all_cars.csv")
    dic1 = awesomeScrap(links['link-href'][1])
    df = pd.DataFrame(dic1, index=[0])
    counter = 0
    for row in range(1, len(links)):
      try:
        check_autovit_limit()
        dic2 = awesomeScrap(links['link-href'][row])
        print(str(counter) + ". scraped " + links['link-href'][row])
        counter+=1
        #time.sleep(0.001)
        df = pd.concat([df, pd.DataFrame(dic2, index=[0])], ignore_index=True)
      except:
        with open("/content/gdrive/MyDrive/Colab Notebooks/data_frame_2.csv", "w") as file:
          file.write(df.to_csv(index=False))
    df.to_csv('secondhandCars.csv')
if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
3066. scraped https://www.autovit.ro/anunt/hyundai-veracruz-3-0-dsl-aut-ID7GYLtu.html
3067. scraped https://www.autovit.ro/anunt/dacia-dokker-1-5-ID7GWN3q.html
3068. scraped https://www.autovit.ro/anunt/peugeot-308-sw-1-6-bluehdi-fap-stt-eat6-ID7H1iJe.html
3069. scraped https://www.autovit.ro/anunt/bmw-seria-5-530d-xdrive-aut-ID7H6DLs.html
3070. scraped https://www.autovit.ro/anunt/audi-a6-ID7H6YIa.html
3071. scraped https://www.autovit.ro/anunt/land-rover-range-rover-3-0l-sdv6-ID7H5fQ1.html
3072. scraped https://www.autovit.ro/anunt/opel-astra-ID7H6QRo.html
3073. scraped https://www.autovit.ro/anunt/skoda-roomster-1-4-tdi-ID7H6PPW.html
3074. scraped https://www.autovit.ro/anunt/volvo-v60-d4-ID7H75Bk.html
3075. scraped https://www.autovit.ro/anunt/renault-captur-tce-gpl-ID7H4ftK.html
3076. scraped https://www.autovit.ro/anunt/seat-exeo-st-2-0-tdi-ID7H3nIP.html
3077. scraped https://www.autovit.ro/anunt/bmw-x5-xdrive30d-ID7GPo9S.html
30

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install ratelimit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5909 sha256=7bb526bcfce1ae0ee02ca21f3e6610d06c95982d20b2d1934220ab3662c81737
  Stored in directory: /root/.cache/pip/wheels/8e/34/5e/1dd3d652594bdf5df01109a683a455121d0b726978051bf720
Successfully built ratelimit
